In [2]:
import glob
import json
import os
import csv
import zstandard as zstd
import io

KEYWORDS = [
    "Occupy", 
    "Occupy Wall Street", 
    "OWS", 
    "corporate greed", 
    "economic inequality", 
    "wealth disparity", 
    "social justice", 
    "protest"
]

SELECTED_COLUMNS = [
    "id", "author", "created_utc", "title", "selftext", "score",
    "num_comments", "subreddit", "permalink", "author_flair_text"
]

In [3]:
def is_relevant(obj, keywords=KEYWORDS):
    text = ""
    for field in ["title", "selftext", "body"]:
        if field in obj and obj[field]:
            text += " " + str(obj[field])
    text = text.lower()
    return any(keyword.lower() in text for keyword in keywords)

def process_zst_files(input_folder, output_csv_zst):
    zst_files = glob.glob(os.path.join(input_folder, '*.zst'))
    if not zst_files:
        print("No .zst files found in the folder.")
        return

    temp_csv = output_csv_zst + '.tmp'
    
    with open(temp_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=SELECTED_COLUMNS, extrasaction='ignore')
        writer.writeheader()  # Write header once

        for zst_file in zst_files:
            print(f"Processing file: {zst_file}")
            with open(zst_file, 'rb') as f:
                dctx = zstd.ZstdDecompressor(max_window_size=2147483648)
                with dctx.stream_reader(f) as reader:
                    text_stream = io.TextIOWrapper(reader, encoding='utf-8')
                    for line in text_stream:
                        try:
                            obj = json.loads(line)
                        except Exception as e:
                            print(f"Error decoding line in {zst_file}: {e}")
                            continue
                        
                        if not is_relevant(obj):
                            continue

                        filtered_obj = {col: obj.get(col, None) for col in SELECTED_COLUMNS}
                        writer.writerow(filtered_obj)

    with open(temp_csv, 'rb') as input_f, open(output_csv_zst, 'wb') as output_f:
        cctx = zstd.ZstdCompressor()
        cctx.copy_stream(input_f, output_f)

    os.remove(temp_csv)
    print(f"Compressed CSV written to {output_csv_zst}")

In [4]:
input_folder = 'submissions'
output_csv_zst = 'submissions.csv.zst'
process_zst_files(input_folder, output_csv_zst)

Processing file: submissions/RS_2011-08.zst
Processing file: submissions/RS_2011-09.zst
Processing file: submissions/RS_2011-10.zst
Compressed CSV written to submissions.csv.zst


In [5]:
import pandas as pd
import zstandard as zstd
import io

def read_csv_zst(file_path):
    with open(file_path, 'rb') as compressed_file:
        dctx = zstd.ZstdDecompressor()
        with dctx.stream_reader(compressed_file) as reader:
            return pd.read_csv(io.BytesIO(reader.read()))

# Usage
df = read_csv_zst("submissions.csv.zst")

In [6]:
df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
0,j669q,XoKillRoyJaneoX,1312243163,Should I be mad about this?,I'll formulate a TL;DR after I'm done writing ...,0,28,AskReddit,/r/AskReddit/comments/j669q/should_i_be_mad_ab...,NaN
1,j667w,VictoriousPR,1312243043,How do I uninstall Internet Explorer?,I've seen three ways to do it and none of them...,4,11,AskReddit,/r/AskReddit/comments/j667w/how_do_i_uninstall...,NaN
2,j6656,sexicorsetman,1312242842,"Reddit, I have been realizing lately that my j...",NaN,1,0,AskReddit,/r/AskReddit/comments/j6656/reddit_i_have_been...,NaN
3,j664q,mesrianilaw11,1312242818,Tow Truck Plows into Yoshinoya Restaurant,NaN,1,0,reddit.com,/r/reddit.com/comments/j664q/tow_truck_plows_i...,NaN
4,j664o,majesticartax,1312242813,At what age did you start looking at porn? And...,"i was 11, caught at 12 because i didn't clear ...",2,19,AskReddit,/r/AskReddit/comments/j664o/at_what_age_did_yo...,NaN


In [7]:
import pandas as pd

subreddits = [
    "politics",
    "Ask_Politics",
    "OccupyWallStreet",
    "socialjustice",
    "conspiracy"
]

In [8]:
filtered_df = df[df["subreddit"].isin(subreddits)]

filtered_df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
15,j65pa,Josephat,1312241843,The House vote to raise the debt ceiling shows...,NaN,1,1,politics,/r/politics/comments/j65pa/the_house_vote_to_r...,NaN
35,j6517,SilentNick3,1312240388,Debt deal passes House; Rep. Giffords shows up...,NaN,1,1,politics,/r/politics/comments/j6517/debt_deal_passes_ho...,NaN
109,j62hz,perkyN405,1312235160,"$230,000 For a Guard Dog: Why the Wealthy Are ...",NaN,30,10,politics,/r/politics/comments/j62hz/230000_for_a_guard_...,NaN
125,j61qw,[deleted],1312233487,Protesters disrupt House during debate on debt...,NaN,35,17,politics,/r/politics/comments/j61qw/protesters_disrupt_...,NaN
244,j5y7c,FunDipper,1312226617,Harry Reid: The Debt Ceiling Deal Shows The Se...,NaN,1,0,politics,/r/politics/comments/j5y7c/harry_reid_the_debt...,NaN


In [9]:
filtered_df = filtered_df[filtered_df["author"] != "[deleted]"]

filtered_df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
15,j65pa,Josephat,1312241843,The House vote to raise the debt ceiling shows...,NaN,1,1,politics,/r/politics/comments/j65pa/the_house_vote_to_r...,NaN
35,j6517,SilentNick3,1312240388,Debt deal passes House; Rep. Giffords shows up...,NaN,1,1,politics,/r/politics/comments/j6517/debt_deal_passes_ho...,NaN
109,j62hz,perkyN405,1312235160,"$230,000 For a Guard Dog: Why the Wealthy Are ...",NaN,30,10,politics,/r/politics/comments/j62hz/230000_for_a_guard_...,NaN
244,j5y7c,FunDipper,1312226617,Harry Reid: The Debt Ceiling Deal Shows The Se...,NaN,1,0,politics,/r/politics/comments/j5y7c/harry_reid_the_debt...,NaN
348,j5v1n,Aprivateeye,1312220832,"Behind the Mask... ""Expect Us""",Here's something that came to mind the other d...,6,2,conspiracy,/r/conspiracy/comments/j5v1n/behind_the_mask_e...,NaN


In [10]:
import pandas as pd
import zstandard as zstd
import os

output_csv_zst = "submissions.csv.zst"

temp_csv = "filtered_subreddits.csv"
filtered_df.to_csv(temp_csv, index=False, encoding="utf-8")

with open(temp_csv, 'rb') as input_f, open(output_csv_zst, 'wb') as output_f:
    cctx = zstd.ZstdCompressor()
    cctx.copy_stream(input_f, output_f)

# Remove the temporary CSV file
os.remove(temp_csv)

print(f"Compressed CSV saved as {output_csv_zst}")

Compressed CSV saved as submissions.csv.zst
